# How to run

You can run each cell in order with shift+enter, or run the whole thing at once by using the Kernel menu up above and hitting 'Restart & Run All'

This notebook expects a folder structure as follows
```
---
 | powers.ipynb
 |-data\Orcus - Powers.xlsx
 |-output
```

and will take data\Orcus - Powers.xlsx and create a markdown file output\powers.md

We use Pandas to read in the excel spreadsheet, and define a function which takes a row in the Pandas DataFrame and puts all the values in the right place (with a little logic to handle fields which aren't always there) to create a markdown formatted string.

It's pretty similar to what the existing Mail Merge does, but we have complete control over how we build the string and map values. The key Pythonic bit is the curly braces {} and the `**dict`. Curly braces indicate a place that we want to input a value in a string with a **key**, and ** takes a dictionary and puts the values with the proper keys in the proper place.  

```
'{hello}'.format(**{'hello':'world', 'not_needed':'mars'})
evaluates to
'world'
```


Then we join that list of strings and save a file.  For more on Python, I recommend [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/)

In [1]:
# cell 0, standard imports

import pandas as pd
import os

dot  = '●'

In [2]:
#cell 1, read in data from filename

filename = 'Orcus - Powers.xlsx'

#by default Pandas loads missing values with ugly NaNs, so we replace those with the empty string ''
data = pd.read_excel(os.path.join('data', filename)).fillna('')
data.head()

,Not in Core,Source,List,Name,Flavor,Category,Frequency,Tier,Tags,Action,...,Miss,Effect,Special,Boost,Key Ability,Secondary Ability,Tradition,Details,Maintain Action,Maintain Text
0,,Advanced Combat,Advanced Combat,Chapter,,,,0,,,...,,,,,,,,,,
1,,Advanced Combat,Advanced Combat,Damage Power Source,,Attack,At-Will,,"Martial, Weapon",Standard,...,,,,,,,,,,
2,,Advanced Combat,Advanced Combat,Disarm,,Attack,At-Will,,"Martial, Weapon",Standard,...,,You provoke an attack of opportunity from the ...,A disarmed creature cannot use powers with the...,,,,,,,
3,,Advanced Combat,Advanced Combat,Duck for Cover,,Utility,At-Will,,Martial,Counter,...,,You fall prone and have superior cover against...,,,,,,,,
4,,Advanced Combat,Advanced Combat,Goad,,Attack,At-Will,,"Focus, Martial",Standard,...,,,,,,,,,,


In [3]:
#cell 2, examine 5 random powers

data.sample(5)

,Not in Core,Source,List,Name,Flavor,Category,Frequency,Tier,Tags,Action,...,Miss,Effect,Special,Boost,Key Ability,Secondary Ability,Tradition,Details,Maintain Action,Maintain Text
494,,Discipline,Puppeteer’s String,Indirect Camouflage,The best disguise is the one established in th...,Utility,Daily,2,"Arcane, Consistent, Focus, Illusion, Psychic",Move,...,,,,,,,,,Move,The effect persists. You can maintain until th...
448,,Discipline,Mastermind’s Gambit,Hearten The Squad,There stands the enemy and your group must des...,Attack,Daily,15,"Focus, Martial",Move,...,"Spend a recovery, but regain no hit points; in...",,,,,,,,,
104,,Discipline,Art of War,Reflex Shot,Your training creates an instinctual reaction ...,Attack,Encounter,13,"Martial, Weapon",Counter,...,,"Make a basic attack against the target, and th...",,,,,,,,
233,,Discipline,"Deep, Dark, Truthful Mirror",Instinctive Shift,You change form without conscious thought.,Utility,Daily,10,"Arcane, Transmutation",Counter,...,,You can use a power with the Transmutation tag...,,,,,,,,
261,,Discipline,Elemental Flux,Cascade of Elemental Wrath,Each slash of your blade releases more and mor...,Attack,Daily,25,"Arcane, Flux, Weapon",Standard,...,,,,,,,,,,


In [4]:
#cell 3
#we define a function which takes in a row of this power dataframe and returns an md string

def power_to_md(row):
    
    #converting the row to a dictionary let's use do some cool stuff with placing values in strings using .format(**row_dict)
    row_dict = dict(row)
    
    #remove leading and trailing whitespace
    for k, v in row_dict.items():
        row_dict[k] = str(v).strip()
    
    # we use completion to check a few edge cases
    completion = dict()
    for key,value in row.items():
        if value:
            completion[key] = value
    
    if row_dict.get('Name') == 'Chapter':
        md = '## {List}  '.format(**row_dict)
        if row['Key Ability']:
            md += "\n**Key Ability:** {Key Ability}  ".format(**row_dict)
        if row['Secondary Ability']:
            md += "\n**Secondary Ability:** {Secondary Ability}  ".format(**row_dict)
        if row['Details']:
            md += "\n\n{Details}  ".format(**row_dict)
        return md
    
   
    # we check if nothing is filling in and return an empty row
    if list(completion.keys()) == []:
        md = ''
        return md
    
    #start with name. md is the string we'll be outputing
    md = "> #### {Name}  ".format(**row_dict)
          
    if row['Flavor']:
        md += '\n> *{Flavor}*  '.format(**row_dict)
        
    #modify tags a little, since they aren't always present    
    if row_dict.get('Tags'):
        row_dict['Tags'] = '● **{Tags}**'.format(**row_dict)
    
    #this is a complex chunk
    md += """\n> **{Frequency}** **{Category}** **{Tier}** (**{Action} Action**) {Tags}  """.format(**row_dict)
    
    if row['Range']:
        md += '\n> **{Range}** {Range Details}  '.format(**row_dict)
    
    #if a field exists, we add it
    if row['Requirement']:
        md += '\n> **Requirements** {Requirement}  '.format(**row_dict)
        
    if row['Trigger']:
        md += '\n> **Trigger** {Trigger}  '.format(**row_dict)
        
    if row['Attack']:
        md += '\n> **Attack** {Attack} vs {Defense}  '.format(**row_dict)
        
    if row['Hit']:
        md += '\n> **Hit** {Hit}  '.format(**row_dict)
    
    if row['Miss']:
        md += '\n> **Miss** {Miss}  '.format(**row_dict)
        
    if row['Effect']:
        md += '\n> **Effect** {Effect}  '.format(**row_dict)
        
    if row['Special']:
        md += '\n> **Special** {Special}  '.format(**row_dict)
        
    if row['Maintain Action']:
        md += '\n> **Maintain {Maintain Action}** {Maintain Text}  '.format(**row_dict)
        
    if row['Boost']:
        md += '\n> **Boost** {Boost}  '.format(**row_dict)
    
    #finally we remove the empty tags
    return md.replace('****','')


In [5]:
#cell 4
#this cell converts the dataframe into a list of markdown formatted strings using the function defined above

output = []

for idx, row in data.iterrows():
    md = power_to_md(row)
    if md: #this removes empty lines, which you may not want
        output.append(md)

In [6]:
#cell 5
#we can look at elements here using Python list slicing

for elem in output[100:110]:
    print(elem)
    print('')

> #### Faster Than The Hammer  
> *You hear your enemy’s muscles twitch as they draw back their weapon.*  
> **Encounter** **Attack** **3** (**Counter Action**) ● **Martial, Weapon**  
> **Melee or Ranged** weapon, the triggering creature  
> **Trigger** A creature makes a ranged attack against you.  
> **Effect** Make a basic attack against the target.  The target suffers a -4 penalty to the triggering attack roll.  

> #### Flow or Crash  
> *Become what you need to survive. Adapt, flow, slide, and prove yourself.*  
> **Encounter** **Utility** **6** (**Free Action**) ● **Martial**  
> **Self**   
> **Effect** You gain a move action.  

> #### False Strike  
> *Your first strike was intentionally lackluster to trick your opponent into a mistake. You counter and strike hard before they realize their folly.*  
> **Encounter** **Attack** **7** (**Standard Action**) ● **Martial, Weapon**  
> **Melee** weapon, one creature  
> **Attack** Strength vs AC  
> **Hit** 3dW + Strength modifier 

In [7]:
#cell 6
# now we have to save to disk

fname =  'Orcus Powers.md'

with open(os.path.join('output', fname), 'w', encoding='utf-8') as f:
    f.write('\n\n'.join(output))